### Labrador Embeddings

In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from huggingface_hub import hf_hub_download
import json

from src.preprocessing import preprocess_df, TextEncoder
from sklearn.model_selection import train_test_split

from src.labrador import Labrador
from src.tokenizers import LabradorTokenizer
from src.dataset import LabradorDataset

from src.train import train_labrador
from src.test import test_model

### Constants

In [2]:
# Dataset:
FILE = 'data/morning_lab_values.csv'
COLUMNS = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']

# Device:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('mps') if torch.cuda.is_available() else torch.device('cpu') # Apple Silicon

# Data loader: 
test_size = 0.2
batch_size = 512
num_workers = 4
max_len = 8

# Model:
embedding_dim = 756
hidden_dim = 756
transformer_heads = 4 #12
num_blocks = 10 #12
transformer_feedforward_dim = 1024 #3072
dropout_rate = 0.3
continuous_head_activation = 'relu'

# Training:
model_path = 'labrador_model.pth'

### Read dataset

In [3]:
df = pd.read_csv(FILE)
df.head()

,hadm_id,subject_id,itemid,charttime,charthour,storetime,storehour,chartday,valuenum,cnt
0,NaN,10312413,51222,2173-06-05 08:20:00,8,2173-06-05 08:47:00,8,2173-06-05,12.8,8
1,25669789.0,10390828,51222,2181-10-26 07:55:00,7,2181-10-26 08:46:00,8,2181-10-26,9.4,8
2,26646522.0,10447634,51222,2165-03-07 06:55:00,6,2165-03-07 07:23:00,7,2165-03-07,11.1,8
3,27308928.0,10784877,51222,2170-05-11 06:00:00,6,2170-05-11 06:43:00,6,2170-05-11,10.3,8
4,28740988.0,11298819,51222,2142-09-13 07:15:00,7,2142-09-13 09:23:00,9,2142-09-13,10.2,8


### Preprocessing

In [4]:
# MinMaxScaler by default
mrl = preprocess_df(df, columns_to_scale=COLUMNS)

### Generate Sequences

In [5]:
# Generate the sequences:
text_encoder = TextEncoder(Repetition_id=True, labs_as_num=True, return_lists=True)
mrl, grouped_mrl = text_encoder.encode_text(mrl)

In [6]:
mrl.head(3)

itemid,subject_id,hadm_id,chartday,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc,nstr,lab_ids,lab_values
0,10000032,22595853.0,2180-05-07,0.530612,0.007895,0.258621,0.609524,0.088028,0.585253,0.027731,0.004782,Bic 0.5306122448979591 Crt 0.00789473684210526...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.5306122448979591, 0.007894736842105262, 0.2..."
1,10000032,22841357.0,2180-06-27,0.469388,0.007895,0.318966,0.504762,0.102113,0.571429,0.055462,0.007515,Bic 0.46938775510204084 Crt 0.0078947368421052...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.46938775510204084, 0.007894736842105262, 0...."
2,10000032,25742920.0,2180-08-06,0.489796,0.015789,0.413793,0.504762,0.130282,0.557604,0.053782,0.008539,Bic 0.48979591836734687 Crt 0.0157894736842105...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.48979591836734687, 0.015789473684210523, 0...."


In [7]:
# See the largest element in: 
grouped_mrl.head(3)

,hadm_id,nstr,lab_ids,lab_values
0,20000019.0,[Bic 0.4489795918367347 Crt 0.0289473684210526...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.4489795918367347, 0.02894736842105263, 0.17..."
1,20000024.0,[Bic 0.46938775510204084 Crt 0.028947368421052...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.46938775510204084, 0.02894736842105263, 0.3..."
2,20000034.0,[Bic 0.4489795918367347 Crt 0.0605263157894736...,"[Bic, Crt, Pot, Sod, Ure, Hgb, Plt, Wbc]","[0.4489795918367347, 0.06052631578947368, 0.28..."


#### Train and Test Split

In [8]:
lab_ids = grouped_mrl.lab_ids.values
lab_values = grouped_mrl.lab_values.values

lab_ids_train, lab_ids_test, lab_values_train, lab_values_test = train_test_split(lab_ids, lab_values, test_size=test_size, random_state=42)

#### Load Tokenizer

In [9]:
repo_id = "dsrestrepo/Labrador_pt"
model_filename = "labrador_model.pth"
tokenizer_config_filename = "tokenizer_config.json"

model_path = hf_hub_download(repo_id=repo_id, filename=model_filename)
tokenizer_config_path = hf_hub_download(repo_id=repo_id, filename=tokenizer_config_filename)

In [10]:
# Load the tokenizer configuration
with open(tokenizer_config_path, 'r') as f:
    tokenizer_config = json.load(f)

# Assuming 'LabradorTokenizer' is your tokenizer class
tokenizer = LabradorTokenizer()
tokenizer.vocab = tokenizer_config['vocab']
tokenizer.mask_token = tokenizer_config['mask_token']
tokenizer.null_token = tokenizer_config['null_token']
tokenizer.pad_token = tokenizer_config['pad_token']

#### Dataloader

In [11]:
dataset_test = LabradorDataset(continuous=lab_values_test, categorical=lab_ids_test, tokenizer=tokenizer, max_len=max_len, masking_prob=0)
# Dataloader:
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

{'[MASK]': 8, '[NULL]': 9, '[PAD]': 10, 'Crt': 0, 'Wbc': 1, 'Hgb': 2, 'Plt': 3, 'Ure': 4, 'Sod': 5, 'Pot': 6, 'Bic': 7}


### Load Model

In [12]:
mask_token = tokenizer.mask_token #-1
null_token = tokenizer.null_token #-2
pad_token = tokenizer.pad_token #-3
vocab_size = tokenizer.vocab_size()

In [13]:
model = Labrador(mask_token=mask_token, pad_token=pad_token, null_token=null_token, vocab_size=vocab_size, embedding_dim=embedding_dim, transformer_heads=transformer_heads, num_blocks=num_blocks, transformer_feedforward_dim=transformer_feedforward_dim, include_head=True, continuous_head_activation=continuous_head_activation, dropout_rate=dropout_rate)
model.load_state_dict(torch.load(model_path))
model.eval()

Labrador(
  (categorical_embedding_layer): Embedding(14, 756)
  (continuous_embedding_layer): ContinuousEmbedding(
    (special_token_embeddings): Embedding(3, 756)
    (dense1): Linear(in_features=1, out_features=756, bias=True)
    (dense2): Linear(in_features=756, out_features=756, bias=True)
    (layernorm): LayerNorm((756,), eps=1e-05, elementwise_affine=True)
  )
  (projection_layer): Linear(in_features=1512, out_features=756, bias=True)
  (blocks): ModuleList(
    (0-9): 10 x TransformerBlock(
      (att): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=756, out_features=756, bias=True)
      )
      (ffn): Sequential(
        (0): Linear(in_features=756, out_features=1024, bias=True)
        (1): ReLU()
        (2): Linear(in_features=1024, out_features=756, bias=True)
      )
      (layernorm1): LayerNorm((756,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((756,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dr

### Test the model

In [14]:
test_model(model, test_loader, device, COLUMNS)

Evaluating Bic: 


  0%|          | 0/104 [00:00<?, ?it/s]

  3%|▎         | 3/104 [00:00<00:24,  4.19it/s]

Preds: [0.514218807220459, 0.5179949998855591, 0.5160105228424072, 0.5181593298912048, 0.5173887014389038, 0.5178527235984802, 0.5167436599731445, 0.5116348266601562, 0.508784830570221, 0.5150989890098572, 0.5155614614486694, 0.5153014659881592, 0.5182420015335083, 0.5134782791137695, 0.5103710889816284, 0.5132918953895569, 0.5167942047119141, 0.5183770060539246, 0.5152066349983215, 0.5121230483055115, 0.5173952579498291, 0.4996969997882843, 0.5160592794418335, 0.5188381671905518, 0.5196415781974792, 0.516944408416748, 0.5199286341667175, 0.5162349343299866, 0.5157026648521423, 0.5191012024879456, 0.5164786577224731, 0.516703724861145, 0.5174960494041443, 0.5181660652160645, 0.5175169110298157, 0.5177950263023376, 0.5159177780151367, 0.5152301788330078, 0.5163831114768982, 0.5182037353515625, 0.5157322883605957, 0.5132775902748108, 0.5176511406898499, 0.5180697441101074, 0.5157209634780884, 0.5178502798080444, 0.5168100595474243, 0.5147024393081665, 0.5152279734611511, 0.51803928613662

100%|██████████| 104/104 [00:06<00:00, 16.33it/s]


...

  3%|▎         | 3/104 [00:00<00:12,  8.30it/s]

Preds: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

100%|██████████| 104/104 [00:05<00:00, 17.52it/s]

RMSE: 0.012
MAE: 0.010
R2: -1.680
-------------------


{'Bic': {'rmse': [0.086016738195594],
  'mae': [0.06613384850597469],
  'r2': [-0.20131242006529027]},
 'Crt': {'rmse': [0.042110525297904704],
  'mae': [0.020773371241305585],
  'r2': [-0.08753747856980088]},
 'Pot': {'rmse': [0.04585034396300611],
  'mae': [0.034253659384675235],
  'r2': [0.012102459024660561]},
 'Sod': {'rmse': [0.06622855158581785],
  'mae': [0.05665971828306621],
  'r2': [-2.086504283292569]},
 'Ure': {'rmse': [0.0637500391866958],
  'mae': [0.03618826496614874],
  'r2': [-0.06523546560134696]},
 'Hgb': {'rmse': [0.09948469824450878],
  'mae': [0.08100428777511412],
  'r2': [-0.10704083890135707]},
 'Plt': {'rmse': [0.049975810210515205],
  'mae': [0.03447125158892357],
  'r2': [-0.1474695135895432]},
 'Wbc': {'rmse': [0.012445089987730659],
  'mae': [0.00985360168744061],
  'r2': [-1.6802015094659697]}}